*This notebook was developed by Marina Ricci for the DESC CL_Cosmo_Pipeline team.*
# This notebook aims at presenting the cluster pipeline for 
# stacked $\Delta\Sigma$ profiles and counts in richness/redshift bins.

It is meant to be run independently. If you already produced the necessary outputs `ceci` will use them, and otherwise create them.

___

In [1]:
import os
from pprint import pprint

import ceci
import h5py

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import yaml
from IPython.display import Image
from astropy.table import Table

import re
import sacc

/global/cfs/projectdirs/lsst/groups/CL/cl_pipeline_project/conda_envs/txpipe_clp/lib/python3.10/site-packages/ceci/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


Make sure to change your path in the next cell that leads to your TXPipe directory. See examples for IN2P3 and NERSC below.

In [2]:
# user specific paths -- IN2P3 example
my_txpipe_dir = "/global/homes/e/edujb/TXPipe/"

# user specific paths -- NERSC example
# my_txpipe_dir = "/pscratch/sd/a/avestruz/TXPipe"

os.chdir(my_txpipe_dir)

import txpipe

___

In [3]:
step1 = txpipe.TXSourceSelectorMetadetect.make_stage(
    # This file is the input metadetect shear catalog
    shear_catalog=f"{my_txpipe_dir}/data/example/inputs/metadetect_shear_catalog.hdf5",
    # This is an input training set for the tomographic selection
    calibration_table=f"{my_txpipe_dir}data/example/inputs/sample_cosmodc2_w10year_errors.dat",

    # This contains all the options for this stage. You can override them here
    # manually too.
    config=f"{my_txpipe_dir}/examples/metadetect/config.yml",

    # This is the output file for this stage
    output_dir=f"{my_txpipe_dir}/data/example/outputs_metadetect/"
)

In [4]:
step1.run()
step1.finalize()

Process 0 running selection for rows 0-82,200


In [5]:
step2 = txpipe.CLClusterBinningRedshiftRichness.make_stage(
    # This file is the input metadetect shear catalog
    cluster_catalog=f"{my_txpipe_dir}data/example/inputs/cluster_catalog.hdf5",
    # This is an input training set for the tomographic selection

    # This contains all the options for this stage. You can override them here
    # manually too.
    config=f"{my_txpipe_dir}/notebooks/cluster_counts/gt_treecor_test/config-1deg2-CL.yml",

    # This is the output file for this stage
    output_dir=f"{my_txpipe_dir}/data/example/output_binning/"
)



        ("binned_shear_catalog", ShearCatalog),
        ("binned_lens_catalog", HDFFile),
        ("binned_random_catalog", HDFFile),
        ("binned_random_catalog_sub", HDFFile),
        ("shear_photoz_stack", QPNOfZFile),
        ("lens_photoz_stack", QPNOfZFile),
        ("patch_centers", TextFile),
        ("tracer_metadata", HDFFile),

IndentationError: unexpected indent (260147577.py, line 16)

In [6]:
step2.run()
step2.finalize()

NameError: name 'step2' is not defined

In [7]:
step3 = txpipe.extensions.cluster_counts.TXTwoPointRLens.make_stage(
    # This file is the input metadetect shear catalog
    binned_lens_catalog=f"global/homes/e/edujb/TXPipe/cluster_catalog_tomography.hdf5",
    # This is an input training set for the tomographic selection
    binned_shear_catalog="global/homes/e/edujb/TXPipe/shear_tomography_catalog.hdf5",
    binned_random_catalog=f"global/homes/e/edujb/TXPipe/cluster_catalog_tomography.hdf5",
    patch_centers=f"global/homes/e/edujb/TXPipe/cluster_catalog_tomography.hdf5",
    tracer_metadata=f"global/homes/e/edujb/TXPipe/cluster_catalog_tomography.hdf5",
    config=f"{my_txpipe_dir}notebooks/cluster_counts/gt_treecor_test/config-1deg2-CL.yml",
    output_dir=f"{my_txpipe_dir}data/example/output_binning/"
)

In [8]:
step3.run()
step3.finalize()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'global/homes/e/edujb/TXPipe/shear_tomography_catalog.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)